<a href="https://colab.research.google.com/github/Suhasri28/My-Calm-Space-Version1/blob/main/Medical_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython langchain faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.2 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [6]:
!pip install biopython

In [7]:
from Bio import Entrez
import time
import warnings
from typing import List, Dict

# Suppress Biopython warnings
warnings.filterwarnings("ignore")

class PubMedChatbot:
    def __init__(self, email: str):
        """
        Initialize the PubMed chatbot.

        Args:
            email: Your email address (required for PubMed API)
        """
        Entrez.email = email  # Required by PubMed API
        self.max_results = 5  # Default number of results to return
        self.max_abstract_length = 500  # Characters to show from abstract

    def search_pubmed(self, query: str, max_results: int = None) -> List[Dict]:
        """
        Search PubMed for articles related to the query.

        Args:
            query: Medical search query
            max_results: Maximum number of results to return (overrides default if provided)

        Returns:
            List of dictionaries containing article information
        """
        if max_results is None:
            max_results = self.max_results

        try:
            # Search PubMed
            handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
            record = Entrez.read(handle)
            handle.close()

            if not record["IdList"]:
                return []

            # Fetch details for the found articles
            id_list = ",".join(record["IdList"])
            handle = Entrez.efetch(db="pubmed", id=id_list, retmode="xml")
            articles = Entrez.read(handle)["PubmedArticle"]
            handle.close()

            # Parse article information
            results = []
            for article in articles:
                article_info = article["MedlineCitation"]["Article"]
                pub_date = article_info.get("Journal", {}).get("JournalIssue", {}).get("PubDate", {})
                year = pub_date.get("Year", "Unknown")

                # Handle abstract which might be a list of sections
                abstract = article_info.get("Abstract", {}).get("AbstractText", ["No abstract available"])
                if isinstance(abstract, list):
                    abstract = " ".join(abstract)

                result = {
                    "title": article_info.get("ArticleTitle", "No title available"),
                    "authors": [author.get("LastName", "") + " " + author.get("Initials", "")
                               for author in article_info.get("AuthorList", [])],
                    "journal": article_info.get("Journal", {}).get("Title", "Unknown journal"),
                    "year": year,
                    "abstract": abstract,
                    "pmid": article["MedlineCitation"]["PMID"]
                }
                results.append(result)

            return results

        except Exception as e:
            print(f"Error searching PubMed: {e}")
            return []

    def generate_summary(self, articles: List[Dict], query: str) -> str:
        """
        Generate a simple summary of the PubMed results without using external NLP.

        Args:
            articles: List of articles from PubMed
            query: Original user query

        Returns:
            Summary string
        """
        if not articles:
            return "No relevant articles found in PubMed for your query."

        # Basic statistics
        num_articles = len(articles)
        years = [int(article["year"]) for article in articles if article["year"].isdigit()]
        min_year = min(years) if years else "unknown"
        max_year = max(years) if years else "unknown"

        # Find common journals
        journals = [article["journal"] for article in articles]
        common_journals = ", ".join(sorted(set(journals), key=lambda x: -journals.count(x))[:3])

        # Find common terms in titles (very simple approach)
        title_words = " ".join(article["title"].lower() for article in articles).split()
        common_terms = ", ".join(sorted(set(
            word for word in title_words
            if len(word) > 4 and word.isalpha() and word not in ["study", "patients", "clinical"]
        ), key=lambda x: -title_words.count(x))[:5])

        summary = (
            f"Found {num_articles} articles related to '{query}' in PubMed.\n"
            f"Publication years range from {min_year} to {max_year}.\n"
            f"Most common journals: {common_journals}\n"
            f"Common terms in titles: {common_terms}\n\n"
            "Top relevant articles:\n"
        )

        for i, article in enumerate(articles[:3], 1):  # Show top 3
            summary += (
                f"\n{i}. {article['title']}\n"
                f"   Authors: {', '.join(article['authors'][:3])}{' et al.' if len(article['authors']) > 3 else ''}\n"
                f"   Journal: {article['journal']} ({article['year']})\n"
                f"   Abstract: {article['abstract'][:self.max_abstract_length]}...\n"
                f"   PMID: {article['pmid']}\n"
            )

        return summary

    def interactive_chat(self):
        """Run an interactive chat session with the PubMed bot."""
        print("PubMed Medical Chatbot (type 'quit' to exit)")
        print("Enter your medical question or topic of interest:")

        while True:
            query = input("\nYou: ")
            if query.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break

            if not query.strip():
                print("Please enter a valid medical question.")
                continue

            print("\nSearching PubMed...")
            articles = self.search_pubmed(query)

            if not articles:
                print("No relevant articles found. Please try a different query.")
                continue

            summary = self.generate_summary(articles, query)
            print(f"\nBot: {summary}")

# Example usage
if __name__ == "__main__":
    # Replace with your email (required by PubMed API)
    bot = PubMedChatbot(email="your.email@example.com")

    # Run interactive chat
    bot.interactive_chat()

PubMed Medical Chatbot (type 'quit' to exit)
Enter your medical question or topic of interest:

You: what are the latest treatment for cancer

Searching PubMed...

Bot: Found 5 articles related to 'what are the latest treatment for cancer' in PubMed.
Publication years range from 2025 to 2025.
Most common journals: Leukemia, Health technology assessment (Winchester, England), Journal of hepato-biliary-pancreatic sciences
Common terms in titles: curative, lymphoma, landscape, diseases, antibacterial

Top relevant articles:

1. Molecular landscape of distinct follicular lymphoma histologic grades: insights from genomic and transcriptome analyses.
   Authors: Sun C, Li W, Yu J et al.
   Journal: Leukemia (2025)
   Abstract: The 2022 World Health Organization Classification of Haematolymphoid tumours classifies follicular lymphoma grades 1-2 (FL1-2) and grade 3A (FL3A) as classic follicular lymphoma (cFL) and reclassifies grade 3B (FL3B) as follicular large B-cell lymphoma (FLBL), without a